## validated
- inserted LSTM
    - accuracy tested 92
- inserted BasicNeatCell
    - accuracy tested 89
- inserted AdvancedNeatCell
    - accuracy tested 94


## built
- create another metric for cells (accuracy)
- create another metric for cells (mse/rmse)
- create another metric for cells (confusion matrix mean)

 
## in-progress
- create a faster training method



## backlog



In [1]:
import os
os.chdir(os.path.dirname('/home/hackerman/GitHub/exploration/tf_fun/'))

# propriatary mnist management
DATASETSLIB_HOME = './datasetslib'
import sys
if not DATASETSLIB_HOME in sys.path:
    sys.path.append(DATASETSLIB_HOME)
%reload_ext autoreload
%autoreload 2
import datasetslib

from datasetslib import util as dsu
datasetslib.datasets_root = os.path.join(os.path.expanduser('~'),'datasets')

/home/hackerman/anaconda2/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(os.path.join(datasetslib.datasets_root,'mnist'),one_hot=True)

Extracting /home/hackerman/datasets/mnist/train-images-idx3-ubyte.gz
Extracting /home/hackerman/datasets/mnist/train-labels-idx1-ubyte.gz
Extracting /home/hackerman/datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting /home/hackerman/datasets/mnist/t10k-labels-idx1-ubyte.gz


In [3]:
num_units = 8
n_classes = 10
batch_size=100
learning_rate = 0.01
learning_rate2 = 0.001
epochs = 10000

In [4]:
timesteps = 28
n_inputs = 28

In [5]:
import tensorflow as tf
tf.reset_default_graph()

In [6]:
os.chdir(os.path.dirname('/home/hackerman/GitHub/NAS_CEPTION/'))
#from modifiedLSTM import AdvancedNeatCell 
#cell = AdvancedNeatCell(num_units)


from modifiedLSTM import BasicNeatCell 
cell = BasicNeatCell(num_units)


# cell = tf.contrib.rnn.BasicLSTMCell(num_units)

# to good for its own good, not using it
# cell = tf.contrib.rnn.GRUCell(num_units)

In [7]:


#                                               num of inputs,size of array for input
inputs = tf.placeholder(dtype='float',shape=[None,timesteps,n_inputs],name='inputs')
output = tf.placeholder(dtype='float',shape=[None,n_classes],name='output')

out_weights=tf.Variable(tf.random_normal([num_units,n_classes]),'weight')
out_bias=tf.Variable(tf.random_normal([n_classes]),'out_bias')

inputStack=tf.unstack(inputs,timesteps,1)

# Placeholders to take in batches of data
outputs,_ = tf.nn.static_rnn(cell, inputStack,dtype='float')
prediction=tf.matmul(outputs[-1],out_weights)+out_bias


loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=output))

opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

opt2=tf.train.AdamOptimizer(learning_rate=learning_rate2).minimize(loss)

#model evaluation
acc, acc_op = tf.metrics.accuracy(labels=tf.argmax(output, 1),       
                                  predictions=tf.argmax(prediction,1))
mse, mse_op = tf.metrics.mean_squared_error(labels=tf.argmax(output, 1),
                                            predictions=tf.argmax(prediction,1))
rmse, rmse_op = tf.metrics.root_mean_squared_error(labels=tf.argmax(output, 1),
                                            predictions=tf.argmax(prediction,1))


#tf_label = tf.placeholder(dtype=tf.int32, shape=[None])
#tf_prediction = tf.placeholder(dtype=tf.int32, shape=[None])

# Define the metric and update operations name="my_metric")

# Isolate the variables stored behind the scenes by the metric operation
#running_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="my_metric")

# Define initializer to initialize/reset running variables
#running_vars_initializer = tf.variables_initializer(var_list=running_vars)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [8]:
iou,iou_op = tf.metrics.mean_iou(labels=tf.argmax(output, 1),
                                            predictions=tf.argmax(prediction,1),num_classes=n_classes)

In [9]:
#initialize variables

with tf.Session() as sess:
    init_g = tf.global_variables_initializer()
    init_l = tf.local_variables_initializer()
    sess.run(init_g)
    sess.run(init_l)

    for epoch in range(epochs):
        batch_x,batch_y=mnist.train.next_batch(batch_size=batch_size)

        batch_x=batch_x.reshape((batch_size,timesteps,n_inputs))
        if epoch < epochs / 2:
            sess.run(fetches=[opt], feed_dict={inputs: batch_x, output: batch_y})
        else:
            sess.run(fetches=[opt2], feed_dict={inputs: batch_x, output: batch_y})

        if epoch % (epochs / 10)==0 or epoch == epochs - 1:
            #acc_ops is needed, othersiwe no tensor to pass to next accuracy
            # test accuracy and mse at same time, allowed
            fetch_opt = [opt]
            fetch_metric_op = [acc_op, mse_op, rmse_op]
            acc_eval, mse_eval, rmse_eval =sess.run(fetches=fetch_metric_op,feed_dict={inputs:batch_x,output:batch_y})
            
            
            # give the confusion matrix
            iou_eval2 = sess.run(fetches=[iou_op],feed_dict={inputs:batch_x,output:batch_y})
            # gives the mean for confusion matrix, must run iou_ops before getting this value
            iou_eval = sess.run(fetches=[iou],feed_dict={inputs:batch_x,output:batch_y})
            
            
            los=sess.run(loss,feed_dict={inputs:batch_x,output:batch_y})
            print("For epoch ",epoch)
            print("Accuracy ",acc_eval)
            print("Mean-Square Error ", mse_eval)
            print("Root Mean-Square Error ", rmse_eval)
            print("Confusion Matrix \n", iou_eval2)
            print("Mean IoU \n", iou_eval)
            print("Loss ",los)
            print("__________________")
            
    
    #calculating test accuracy
    # resetting the accuracy
    sess.run(init_l)
    test_data = mnist.test.images[:128].reshape((-1, timesteps, n_inputs))
    test_label = mnist.test.labels[:128]
    # acc_op needs to be run first because on GPU, doing[acc_op,acc] will be executed in random order
    acc_eval ,iou_eval,mse_eval = sess.run([acc_op,iou_op,mse_op],feed_dict={inputs: test_data, output: test_label})
    miou_eval = sess.run(fetches=[iou],feed_dict={inputs: test_data, output: test_label})
    # acc_eval  = sess.run([acc],feed_dict={inputs: test_data, output: test_label})
    print("Test Batch Accuracy:", acc_eval)
    print("Test Batch Mean IoU:", miou_eval)
    print("Test Batch Mean SqErr ", mse_eval)
    print("Mean IoU \n", iou_eval)

For epoch  0
Accuracy  0.08
Mean-Square Error  29.89
Root Mean-Square Error  5.4671745
Confusion Matrix 
 [array([[ 8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 9.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [12.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [15.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 9.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 9.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [14.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])]
Mean IoU 
 [0.007999999]
Loss  2.7690628
__________________
For epoch  1000
Accuracy  0.36
Mean-Square Error  17.46
Root Mean-Square Error  4.1785164
Confusion Matrix 
 [array([[15.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.],
       [ 9.,  6.,  0.,  0.,  3.,  0.,  0.,  0.,  0.,  0.],
       [ 8., 

Test Batch Accuracy: 0.8046875
Test Batch Mean IoU: [0.6693318]
Test Batch Mean SqErr  1.8359375
Mean IoU 
 [[10.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 15.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  9.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  8.  0.  4.  0.  0.  0.  0.]
 [ 0.  1.  1.  0. 14.  1.  0.  0.  1.  2.]
 [ 0.  0.  0.  7.  0.  1.  0.  1.  1.  0.]
 [ 0.  0.  0.  0.  0.  0. 12.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. 18.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  2.  0.]
 [ 0.  0.  0.  0.  2.  0.  0.  1.  0. 14.]]


In [10]:
'''
Test Batch Accuracy: 0.734375
Test Batch Mean IoU: [0.57026863]
Test Batch Mean SqErr  3.390625
Mean IoU 
 [[ 9.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0. 15.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  8.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  6.  0.  2.  1.  0.  2.  1.]
 [ 0.  1.  1.  0. 17.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  8.  1.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0. 12.  0.  0.  0.]
 [ 0.  1.  0.  1.  1.  0.  0. 13.  0.  3.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  2.  0.]
 [ 0.  0.  0.  0.  5.  0.  0.  0.  0. 12.]]

'''


'\nTest Batch Accuracy: 0.734375\nTest Batch Mean IoU: [0.57026863]\nTest Batch Mean SqErr  3.390625\nMean IoU \n [[ 9.  0.  0.  1.  0.  0.  0.  0.  0.  0.]\n [ 0. 15.  0.  0.  0.  0.  0.  0.  0.  0.]\n [ 1.  0.  8.  0.  0.  0.  0.  0.  1.  0.]\n [ 0.  0.  0.  6.  0.  2.  1.  0.  2.  1.]\n [ 0.  1.  1.  0. 17.  0.  0.  0.  0.  1.]\n [ 0.  0.  0.  8.  1.  0.  0.  0.  1.  0.]\n [ 0.  0.  0.  0.  0.  0. 12.  0.  0.  0.]\n [ 0.  1.  0.  1.  1.  0.  0. 13.  0.  3.]\n [ 0.  0.  0.  0.  0.  0.  1.  0.  2.  0.]\n [ 0.  0.  0.  0.  5.  0.  0.  0.  0. 12.]]\n\n'

In [ ]:
'''
Basic Neat Cell
Test Batch Accuracy: 0.8046875
Test Batch Mean IoU: [0.6693318]
Test Batch Mean SqErr  1.8359375
Mean IoU 
 [[10.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 15.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  9.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  8.  0.  4.  0.  0.  0.  0.]
 [ 0.  1.  1.  0. 14.  1.  0.  0.  1.  2.]
 [ 0.  0.  0.  7.  0.  1.  0.  1.  1.  0.]
 [ 0.  0.  0.  0.  0.  0. 12.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. 18.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  2.  0.]
 [ 0.  0.  0.  0.  2.  0.  0.  1.  0. 14.]]
 
 '''

In [11]:
'''
Advanced Neat Cell
Test Batch Accuracy: 0.953125
Test Batch Mean IoU: [0.8898122]
Test Batch Mean SqErr  0.8359375
Mean IoU 
 [[10.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 15.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. 10.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 11.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 20.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  9.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. 12.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. 19.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  2.  0.]
 [ 0.  0.  0.  0.  2.  1.  0.  0.  0. 14.]]
'''

'\nAdvanced Neat Cell\nTest Batch Accuracy: 0.953125\nTest Batch Mean IoU: [0.8898122]\nTest Batch Mean SqErr  0.8359375\nMean IoU \n [[10.  0.  0.  0.  0.  0.  0.  0.  0.  0.]\n [ 0. 15.  0.  0.  0.  0.  0.  0.  0.  0.]\n [ 0.  0. 10.  0.  0.  0.  0.  0.  0.  0.]\n [ 0.  0.  0. 11.  0.  1.  0.  0.  0.  0.]\n [ 0.  0.  0.  0. 20.  0.  0.  0.  0.  0.]\n [ 0.  0.  0.  0.  0.  9.  1.  0.  0.  0.]\n [ 0.  0.  0.  0.  0.  0. 12.  0.  0.  0.]\n [ 0.  0.  0.  0.  0.  0.  0. 19.  0.  0.]\n [ 0.  0.  1.  0.  0.  0.  0.  0.  2.  0.]\n [ 0.  0.  0.  0.  2.  1.  0.  0.  0. 14.]]\n'